# Method (1)

In [1]:
import numpy as np
import pandas as pd

Preprocessing step

I first need to do the data cleaning the details of steps are below :

(1)Load the json and csv file - with pd.read_json and pd.read_csv

(2)Combine the Dataset - with pd.merge

(3)Text cleaning - with cleanTxt

(4)Create the dataset  - with to_pickle

(5)Split the training dataset into train and validation set - with train_test_split

In [9]:
# Prepare the train.csv and test.csv
df =pd.read_csv('data_identification.csv')
train_df = df.loc[df['identification']=='train']
test_df = df.loc[df['identification']=='test']

header = ['tweet_id', 'identification']
train_df.to_csv('train.csv', index = False, columns = header)
test_df.to_csv('test.csv', index = False, columns = header)

In [10]:
print(len(train_df), len(test_df))

1455563 411972


In [11]:
#Load the json file 
try_df = pd.read_json('tweets_DM.json', lines=True)

input_dict = list(pd.DataFrame(list(try_df._source.values)).tweet)
df_dict = pd.DataFrame.from_dict(input_dict)
df_dict.head(5)

,hashtags,tweet_id,text
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,[],0x2de201,"""Trust is not the same as faith. A friend is s..."


In [12]:
#CLean the text
import re
def cleanTxt(text):
    text = re.sub(r'@[A-Za-z0-9]+','',text)
    text = re.sub(r'RT[\s]+', '', text)
    text = re.sub(r'http?:\/\/\S+','',text)
    text = re.sub(r'#', '', text)
    
    return text

In [27]:
#Cleaning the text
df_dict['text'] = df_dict['text'].apply(cleanTxt)
df_dict.head(5)

,hashtags,tweet_id,text
0,[Snapchat],0x376b20,"People who post ""add me on Snapchat"" must be d..."
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"As we see, Trump is dangerous to freepress ar..."
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,[],0x2de201,"""Trust is not the same as faith. A friend is s..."


In [29]:
'''
df = pd.read_json("tweets_DM.json", encoding='utf-8', lines=True)
df_label = pd.read_csv("emotion.csv")

Series_of_dict = list(pd.DataFrame(list(df._source.values)).tweet)
df_data = pd.DataFrame.from_dict(Series_of_dict)
df_data.head()
'''

'\ndf = pd.read_json("tweets_DM.json", encoding=\'utf-8\', lines=True)\ndf_label = pd.read_csv("emotion.csv")\n\nSeries_of_dict = list(pd.DataFrame(list(df._source.values)).tweet)\ndf_data = pd.DataFrame.from_dict(Series_of_dict)\ndf_data.head()\n'

# Combine the Dataset

In [30]:
train_merg_df = pd.merge(train_df, df_dict, how='left', on='tweet_id')
train_merg_df.head()

,tweet_id,identification,hashtags,text
0,0x29e452,train,[],Huge Respect🖒 talking about losing his dad to...
1,0x2b3819,train,"[spateradio, app]",Yoooo we hit all our monthly goals with the ne...
2,0x2a2acc,train,[],_BCH Well done team 🌟 <LH> of every one of ...
3,0x2a8830,train,"[PUBG, GamersUnite, twitch, BeHealthy, StayPos...",Come join on PUBG while he strives for chicke...
4,0x20b21d,train,"[strength, bones, God]",Blessings!My strength little. My bones brittl...


In [31]:
header = ['tweet_id', 'identification']
train_merg_df[['tweet_id', 'text']].head(2)

,tweet_id,text
0,0x29e452,Huge Respect🖒 talking about losing his dad to...
1,0x2b3819,Yoooo we hit all our monthly goals with the ne...


In [32]:
emotion_df=pd.read_csv('emotion.csv')
train_merg_df2 = pd.merge(train_merg_df, emotion_df, how='left', on='tweet_id')
train_merg_df2[['tweet_id', 'text','emotion']].head(2)

,tweet_id,text,emotion
0,0x29e452,Huge Respect🖒 talking about losing his dad to...,joy
1,0x2b3819,Yoooo we hit all our monthly goals with the ne...,joy


In [33]:
test_merg_df = pd.merge(test_df, df_dict, how='left', on='tweet_id')
test_merg_df.head()

,tweet_id,identification,hashtags,text
0,0x28cc61,test,[],I've seen two separate colours of the elegant...
1,0x2db41f,test,[],No serious self respecting individual believ...
2,0x2466f6,test,[womendrivers],"Looking for a new car, and it says 1 lady owne..."
3,0x23f9e9,test,[robbingmembers],“only the brave” just out and fountain park h...
4,0x1fb4e1,test,[],Felt like total dog 💩 going into open gym and ...


In [34]:
train_with_emotion = train_merg_df2[['tweet_id', 'text','emotion']]
train_with_emotion.shape

(1455563, 3)

In [35]:
test_with_emotion = test_merg_df[['tweet_id', 'text']]
test_with_emotion.shape

(411972, 2)

In [36]:
train_with_emotion.to_pickle("train_with_emotion.pkl") 
train_with_emotion_df = pd.read_pickle("train_with_emotion.pkl")

In [37]:
test_with_emotion.to_pickle("test_with_emotion.pkl")
test_with_emotion_df = pd.read_pickle("test_with_emotion.pkl")

# Split the training dataset into train and val dataset

In [38]:
from sklearn.model_selection import train_test_split
#X_train, X_test = train_test_split(train_with_emotion, test_size = 0.3, random_state = 42, shuffle = True)
X_train, X_test = train_test_split(train_with_emotion, test_size = 0.2, random_state = 1, shuffle = True)
X_train.head()

,tweet_id,text,emotion
754714,0x2f11c4,"FailureFriday ....Now , Back to Russia! Impeac...",sadness
105795,0x3072b5,<LH> Holidays,joy
33277,0x1cc8b3,Tonight <LH> 8pm-12MidNite Victory <LH> Chapel,anticipation
723943,0x30c714,Sitting on a plane reflecting on the many lead...,trust
139949,0x2b45aa,Only sikhs have conquered Afghanistan. Many s...,trust


In [39]:
X_test.head()

,tweet_id,text,emotion
764928,0x2ed744,please make the coconut shrimp taco a permane...,joy
1411174,0x255371,Lesson of the Day: you never know when your ki...,trust
1008269,0x1d36fb,What wonderful presentation events we have had...,anticipation
63685,0x238b4f,"<LH> to , graphic design and sign specialists,...",joy
970075,0x262da1,_1 Now there are two idiots! clueless <LH>,sadness


In [40]:
import pickle
## save to pickle file
X_train.to_pickle("model_X_train.pkl") 
X_test.to_pickle("model_X_test.pkl")

In [41]:
import pandas as pd
## load a pickle file
model_X_train_df = pd.read_pickle("model_X_train.pkl")
model_X_test_df = pd.read_pickle("model_X_test.pkl")

# Feature engineering

As learn a lot from lab2, I continute use BOW to do with feature engineering

But this time I set the feature to 1000 to think it can do better


In [42]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
BOW_1000 = CountVectorizer(max_features=1000, tokenizer=nltk.word_tokenize) 

BOW_1000.fit(model_X_train_df['text'])
print("ok")
BOW_features_1000 = BOW_1000.transform(model_X_train_df['text'])
BOW_features_1000.shape

C:\ProgramData\Anaconda3\envs\DM\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


ok


(1164450, 1000)

In [43]:
import keras

# standardize name (X, y) 
X_train = BOW_1000.transform(model_X_train_df['text'])
y_train = model_X_train_df['emotion']

X_test = BOW_1000.transform(model_X_test_df['text'])
y_test = model_X_test_df['emotion']

## check dimension is a good habbit 
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_test.shape)

X_train.shape:  (1164450, 1000)
y_train.shape:  (1164450,)
X_test.shape:  (291113, 1000)
y_test.shape:  (291113,)


Deal with categorical label (y)

In [44]:
## deal with label (string -> one-hot)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:8]:\n', y_train[0:8])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_test = label_encode(label_encoder, y_test)

print('\n\n## After convert')
print('y_train[0:8]:\n', y_train[0:8])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train[0:8]:
 754714          sadness
105795              joy
33277      anticipation
723943            trust
139949            trust
863995          sadness
1131056    anticipation
835483              joy
Name: emotion, dtype: object

y_train.shape:  (1164450,)
y_test.shape:  (291113,)


## After convert
y_train[0:8]:
 [[0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]]

y_train.shape:  (1164450, 8)
y_test.shape:  (291113, 8)


In [45]:
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  1000
output_shape:  8


In [49]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 1000
X = model_input

# 1st hidden layer
X_W1 = Dense(units=64)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=64)(H1)  # 64
H2 = ReLU()(H1_W2)

# 3rd hidden layer
H2_W3 = Dense(units=64)(H2)  # 64
H3 = ReLU()(H2_W3)

# 4th hidden layer
H3_W4 = Dense(units=64)(H3)  # 64
H4 = ReLU()(H3_W4)

# output layer
H5_W6 = Dense(units=output_shape)(H4)  # 4
H5 = Softmax()(H5_W6)

model_output = H5

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1000)]            0         
_________________________________________________________________
dense_9 (Dense)              (None, 64)                64064     
_________________________________________________________________
re_lu_7 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                4160      
_________________________________________________________________
re_lu_8 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 64)                4160      
_________________________________________________________________
re_lu_9 (ReLU)               (None, 64)               

In [46]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=64)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=64)(H1)  # 64
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 4
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1000)]            0         
_________________________________________________________________
dense (Dense)                (None, 64)                64064     
_________________________________________________________________
re_lu (ReLU)                 (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 520       
_________________________________________________________________
softmax (Softmax)            (None, 8)                

In [50]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('logs/training_log.csv')

# training setting
epochs = 100
batch_size = 1500

# training!
model.save("model_clean_f.h5")

history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data = (X_test, y_test))
model.save("model_clean_e.h5")

print('training finish')

Epoch 1/100
777/777 [==============================] - 8s 10ms/step - loss: 1.4621 - accuracy: 0.4667 - val_loss: 1.3938 - val_accuracy: 0.4897
Epoch 2/100
777/777 [==============================] - 8s 11ms/step - loss: 1.3747 - accuracy: 0.4965 - val_loss: 1.3728 - val_accuracy: 0.4973
Epoch 3/100
777/777 [==============================] - 9s 11ms/step - loss: 1.3539 - accuracy: 0.5042 - val_loss: 1.3649 - val_accuracy: 0.4998
Epoch 4/100
777/777 [==============================] - 9s 11ms/step - loss: 1.3412 - accuracy: 0.5088 - val_loss: 1.3608 - val_accuracy: 0.5022
Epoch 5/100
777/777 [==============================] - 9s 11ms/step - loss: 1.3325 - accuracy: 0.5121 - val_loss: 1.3566 - val_accuracy: 0.5037
Epoch 6/100
777/777 [==============================] - 9s 12ms/step - loss: 1.3257 - accuracy: 0.5148 - val_loss: 1.3546 - val_accuracy: 0.5045
Epoch 7/100
777/777 [==============================] - 9s 12ms/step - loss: 1.3202 - accuracy: 0.5173 - val_loss: 1.3548 - val_accuracy:

In [56]:
from keras.models import load_model
M_new_model = load_model("model_clean_e.h5")

In [52]:
# Feature engineering on testing set
import nltk
from sklearn.feature_extraction.text import CountVectorizer
t_BOW_1000 = CountVectorizer(max_features=1000, tokenizer=nltk.word_tokenize) 

t_BOW_1000.fit(test_with_emotion_df['text'])
t_BOW_features_1000 = t_BOW_1000.transform(test_with_emotion_df['text'])
print('ok')
t_BOW_features_1000.shape

C:\ProgramData\Anaconda3\envs\DM\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


ok


(411972, 1000)

In [57]:
pred_result = M_new_model.predict(t_BOW_features_1000, batch_size=3000)

In [58]:
pred_result = label_decode(label_encoder, pred_result)
pred_result[:5]

array(['trust', 'trust', 'trust', 'joy', 'trust'], dtype=object)

In [59]:
from pandas import DataFrame
sub4 = pred_result.tolist()
sub4_df = DataFrame(sub4,columns=['emotion'])
sub4_df.to_csv('sub_model_clean_e33.csv')

# Evaluate the model

In [ ]:
from keras.models import load_model
new_model = load_model("model.h5")

In [ ]:
from keras.models import load_model
new_model = load_model("model.h5")

In [ ]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
t_BOW_1000 = CountVectorizer(max_features=1000, tokenizer=nltk.word_tokenize) 

t_BOW_1000.fit(test_with_emotion_df['text'])

t_BOW_features_1000 = t_BOW_1000.transform(test_with_emotion_df['text'])
print('ok')
t_BOW_features_1000.shape

In [ ]:
pred_result = new_model.predict(t_BOW_features_1000, batch_size=1000)

In [ ]:
new_model.summary()

In [ ]:
new_model.get_weights()

In [ ]:
new_model.optimizer()

In [ ]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
t_BOW_1000 = CountVectorizer(max_features=1000, tokenizer=nltk.word_tokenize) 

t_BOW_1000.fit(test_with_emotion_df['text'])

t_BOW_features_1000 = t_BOW_1000.transform(test_with_emotion_df['text'])
print('ok')
t_BOW_features_1000.shape

In [ ]:
pred_result = model.predict(t_BOW_features_1000, batch_size=1000)

In [ ]:
pred_result = label_decode(label_encoder, pred_result)
pred_result[:5]

In [ ]:
type(pred_result)

In [ ]:
len(pred_result)

In [ ]:
pred_result.tolist()

In [ ]:
sub = pred_result.tolist()

In [ ]:
sub.to_csv('sub.csv')

# END